In [1]:
import cornac
from cornac.eval_methods import RatioSplit, BaseMethod
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from recommenders.evaluation.python_evaluation import serendipity, distributional_coverage, catalog_coverage
from scipy.stats import hmean

from harmonic_mean import HarmonicMean
from serendipity_wrapper import Serendipity
from combined_eval_method import CombinedBaseMethod
from new_random_search import NewRandomSearch

c:\Users\talkt\.conda\envs\cornac2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [2]:
def df_to_tuplelist(df):

    # transform into tuples
    tuple_list = list(df.itertuples(index=False, name=None))

    # rearrange
    for i in range(len(tuple_list)):
        tuple_list[i] = (str(tuple_list[i][1]), str(tuple_list[i][0]), float(tuple_list[i][2]))

    return tuple_list

def df_to_tuplelist_pair(df):
    # make into tuples
    cats_data = list(df.itertuples(index=False, name=None))

    # unzip the tuple into 2 lists
    cat_ids, texts = zip(*cats_data)

    cat_ids = list(cat_ids)
    texts = list(texts)

    for i in range(len(texts)):
        texts[i] = texts[i].replace('\n', ' ')

    return cat_ids, texts

# weighting modifications
def add_weightings(train_df, val_df, test_df):

    # list of datasets
    dfs = [train_df, test_df, val_df]

    for i in range(len(dfs)):
        
        # for all
        dfs[i]['like'] = dfs[i]['like'].apply(lambda x: 1 if x else 0)
        
        # train
        if i == 0:
            dfs[i]['click'] = dfs[i]['click'].apply(lambda x: 1 if x else 0)
            dfs[i].drop(['Unnamed: 0', 'id','created_at', 'updated_at'], axis=1, inplace=True)
            dfs[i] = dfs[i][['userID', 'catID', 'like', 'dwell_time_ms', 'click']]
        
        elif i == 1 or i == 2:
            dfs[i].drop(['Unnamed: 0', 'id', 'dwell_time_ms', 'click', 'created_at', 'updated_at'], axis=1, inplace=True)
            dfs[i] = dfs[i][['userID', 'catID', 'like']]
            
    train, test, val = dfs[0], dfs[1], dfs[2]

    # checking dwell_time_ms in seconds
    train['dwell_time_ms'].apply(lambda x: x/1000).describe()

    # applying log transformation on dwell time to reduce impact of outliers
    train['dwell_time_ms'].apply(lambda x: np.log(x)).describe()

    train['log_dwell_time'] = train['dwell_time_ms'].apply(lambda x: np.log(x))
    train.drop(['dwell_time_ms'], axis=1, inplace=True)

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()

    train['norm_log_dwell_time'] = scaler.fit_transform(train[['log_dwell_time']])
    train.drop(['log_dwell_time'], axis=1, inplace=True)
    train.head()

    train_weighted = train.copy()

    # weights L, C, D for like, click, dwell_time
    L = 0.5
    C = 0.25
    D = 0.25

    train_weighted['rating'] = L*train_weighted['like'] + C*train_weighted['click'] + D* train_weighted['norm_log_dwell_time']

    train_weighted.drop(['like','click','norm_log_dwell_time'], axis=1, inplace=True)

    # convert to tuple
    like_data_train = [tuple(train_weighted.iloc[i]) for i in range(len(train_weighted))]
    like_data_test = [tuple(test.iloc[i]) for i in range(len(test))]
    like_data_validation = [tuple(val.iloc[i]) for i in range(len(val))]

    return like_data_train, like_data_validation, like_data_test

In [3]:
# data-loading

# stratified splits
strat_train_df = pd.read_csv('model_data/train_val_test/strat_train.csv', usecols=['catID', 'userID', 'like'])
strat_validation_df = pd.read_csv('model_data/train_val_test/strat_validation.csv', usecols=['catID', 'userID', 'like'])
strat_test_df = pd.read_csv('model_data/train_val_test/strat_test.csv', usecols=['catID', 'userID', 'like'])

strat_like_data_train = df_to_tuplelist(strat_train_df)
strat_like_data_validation = df_to_tuplelist(strat_validation_df)
strat_like_data_test = df_to_tuplelist(strat_test_df)

# lsuo splits
lsuo_train_df = pd.read_csv('model_data/train_val_test/lsuo_train.csv', usecols=['catID', 'userID', 'like'])
lsuo_validation_df = pd.read_csv('model_data/train_val_test/lsuo_validation.csv', usecols=['catID', 'userID', 'like'])
lsuo_test_df = pd.read_csv('model_data/train_val_test/lsuo_test.csv', usecols=['catID', 'userID', 'like'])

lsuo_like_data_train = df_to_tuplelist(lsuo_train_df)
lsuo_like_data_validation = df_to_tuplelist(lsuo_validation_df)
lsuo_like_data_test = df_to_tuplelist(lsuo_test_df)

#description
desc_cats_df = pd.read_csv('model_data/auxiliary/cats.csv', usecols=['id', 'description'])
desc_cat_ids, desc_texts = df_to_tuplelist_pair(desc_cats_df)

#details
details_cats_df = pd.read_csv('model_data/auxiliary/cats.csv', usecols=['id', 'details'])
details_cat_ids, details_texts = df_to_tuplelist_pair(details_cats_df)

In [4]:
### Function to generate output
def make_recommendations(MODEL, TRAINING_SET):
    item_id2idx = MODEL.train_set.iid_map
    item_idx2id = list(MODEL.train_set.item_ids)
    user_idx2id = MODEL.train_set.uid_map
    user_idx2id = list(MODEL.train_set.user_ids)
   
    num_users = len(np.unique(user_idx2id))
    
    # For each user, get the list of items that they have rated in train and probe
    rated = TRAINING_SET.groupby('userID')['catID'].agg(lambda x: list(x))
    rated = rated.to_dict()

    rec_result = {}

    for UIDX in range(0, num_users):
        recommendations, scores = MODEL.rank(UIDX)
        rec_result[user_idx2id[UIDX]] = [item_idx2id[i] for i in recommendations]

    # sort results
    rec_result = {key:value for key, value in sorted(rec_result.items(), key=lambda item: item[0])}

    # remove the rated items from rec_results
    for user in rec_result:
        tmp = [x for x in rec_result[user] if x not in rated[user]]
        rec_result[user] = tmp

    return rated, rec_result

# Function for generating serendipity, distributional coverage and harmonic mean scores
def evaluate(EXPERIMENT, TRAINING_SET):
    
    MODEL = EXPERIMENT.models[0]
    
    # Get disctionary of rated and recommended items
    rated, recommendations = make_recommendations(MODEL, TRAINING_SET)


    ### PREPARING DATAFRAMES ###
    # Make recommended items into a dataframe for MS recommenders
    data = []
    
    # Iterate over the dictionary keys and values
    for key, values in recommendations.items():
        # Iterate over the values and append rows to the data list
        for value in values:
            data.append([key, value])
    
    # Create the DataFrame with the specified columns
    recommendations_df = pd.DataFrame(data, columns=['userID', 'itemID'])

    # Make the testing set into a dataframe for MS recommenders
    train_df = TRAINING_SET[['userID','catID']].reset_index()
    train_df = train_df[['userID','catID']]
    train_df = train_df.rename(columns={'userID':'userID', 'catID':'itemID'})


    ### METRICS ###
    # Calculate serendipity
    serendipity_score = serendipity(train_df, recommendations_df)

    # Calculate coverage
    dist_coverage_score = distributional_coverage(train_df, recommendations_df)

    # Extract 
    for result in EXPERIMENT.result:
        model_name = result.model_name
        fone10 = result.metric_avg_results['F1@10']
        ncrr = result.metric_avg_results['NCRR@-1']
        ndcg = result.metric_avg_results['NDCG@-1']

    # Calculate harmonic mean
    h_mean = hmean([fone10, ncrr, ndcg, dist_coverage_score, serendipity_score])

    print(model_name)
    print(f"F1@10: {fone10:.3f}")
    print(f"NCRR: {ncrr:.3f}")
    print(f"NDCG: {ndcg:.3f}")
    print(f"Distributional coverage: {dist_coverage_score:.3f}")
    print(f"Serendipity: {serendipity_score:.3f}")
    print(f"Harmonic mean: {h_mean:.3f}")
    print()

In [5]:
def train_ctr(model_name, text_column, split_type, weighted_or_unweighted):

    if weighted_or_unweighted == 'unweighted':
        if split_type == 'strat':
            like_data_train = strat_like_data_train
            like_data_validation = strat_like_data_validation
            like_data_test = strat_like_data_test

        else:
            like_data_train = lsuo_like_data_train
            like_data_validation = lsuo_like_data_validation
            like_data_test = lsuo_like_data_test

    else:
        #junyi's modifications
        # train test val filenames
        trainfile = f'{split_type}_train'
        testfile = f'{split_type}_test'
        valfile = f'{split_type}_validation'

        # train test val datasets
        train = pd.read_csv('model_data/train_val_test/'+trainfile+'.csv')
        test = pd.read_csv('model_data/train_val_test/'+testfile+'.csv')
        val = pd.read_csv('model_data/train_val_test/'+valfile+'.csv')

        # main data dataset
        user = pd.read_csv('model_data/auxiliary/users.csv')
        cat = pd.read_csv('model_data/auxiliary/cats.csv')
        # interaction = pd.read_csv('model_data/auxiliary/interaction.csv')

        # list of datasets
        dfs = [train, test, val]
        for i in range(len(dfs)):
            # for all
            dfs[i]['like'] = dfs[i]['like'].apply(lambda x: 1 if x else 0)
            # train
            if i == 0:
                dfs[i]['click'] = dfs[i]['click'].apply(lambda x: 1 if x else 0)
                dfs[i].drop(['Unnamed: 0', 'id','created_at', 'updated_at'], axis=1, inplace=True)
                dfs[i] = dfs[i][['userID', 'catID', 'like', 'dwell_time_ms', 'click']]
            elif i == 1 or i == 2:
                dfs[i].drop(['Unnamed: 0', 'id', 'dwell_time_ms', 'click', 'created_at', 'updated_at'], axis=1, inplace=True)
                dfs[i] = dfs[i][['userID', 'catID', 'like']]
        train, test, val = dfs[0], dfs[1], dfs[2]
        # checking dwell_time_ms in seconds
        train['dwell_time_ms'].apply(lambda x: x/1000).describe()
        # applying log transformation on dwell time to reduce impact of outliers
        train['dwell_time_ms'].apply(lambda x: np.log(x)).describe()
        train['log_dwell_time'] = train['dwell_time_ms'].apply(lambda x: np.log(x))
        train.drop(['dwell_time_ms'], axis=1, inplace=True)
        scaler = MinMaxScaler()
        train['norm_log_dwell_time'] = scaler.fit_transform(train[['log_dwell_time']])
        train.drop(['log_dwell_time'], axis=1, inplace=True)
        train_weighted = train.copy()
        # weights L, C, D for like, click, dwell_time
        L = 0.5
        C = 0.25
        D = 0.25
        train_weighted['rating'] = L*train_weighted['like'] + C*train_weighted['click'] + D* train_weighted['norm_log_dwell_time']
        train_weighted.drop(['like','click','norm_log_dwell_time'], axis=1, inplace=True)
        # convert to tuple
        like_data_train = [tuple(train_weighted.iloc[i]) for i in range(len(train_weighted))]
        like_data_test = [tuple(test.iloc[i]) for i in range(len(test))]
        like_data_validation = [tuple(val.iloc[i]) for i in range(len(val))]

    if text_column == 'desc':
        cat_ids = desc_cat_ids
        texts = desc_texts

    else:
        cat_ids = details_cat_ids
        texts = details_texts

    # Instantiate a TextModality, it makes it convenient to work with text auxiliary information
    # For more details, please refer to the tutorial on how to work with auxiliary data
    item_text_modality = TextModality(
        corpus=texts,
        ids=cat_ids,
        tokenizer=BaseTokenizer(sep=" ", stop_words="english"),
        max_vocab=8000,
        max_doc_freq=0.5,
    )

    # Define an evaluation method to split feedback into train and test sets

    if split_type == 'strat':
        bm = CombinedBaseMethod.from_splits(
        train_data=like_data_train,
        val_data=like_data_validation,
        test_data=like_data_test,
        verbose=True,
        item_text=item_text_modality,
        exclude_unknowns=True,
        )

    else:
        bm = CombinedBaseMethod.from_splits(
            train_data=like_data_train,
            val_data=like_data_validation,
            test_data=like_data_test,
            verbose=True,
            item_text=item_text_modality,
            exclude_unknowns=False,
            )

    # Instantiate CTR model
    ctr = cornac.models.CTR(
        name=f'{model_name}_{text_column}_{split_type}_{weighted_or_unweighted}',
        k=250,
        max_iter=200,
        lambda_v=1
        )

    # Use these for evaluation
    eval_metrics = [
        HarmonicMean(
            10,
            Serendipity(),
            cornac.metrics.FMeasure(k=10),
            cornac.metrics.NCRR(),
            cornac.metrics.NDCG()
        ),
        Serendipity(),
        cornac.metrics.FMeasure(k=10),
        cornac.metrics.NCRR(),
        cornac.metrics.NDCG()
    ]

    # Wrap MF model inside RandomSearch along with the searching space, try 20 times
    rs_ctr = NewRandomSearch(
        model=ctr,
        space=[
            Discrete("k", [50, 75, 100, 150, 200]),
            Continuous("lambda_u", low=1e-4, high=1e1),
            Continuous("lambda_v", low=1e-4, high=1e1),
            Continuous("a", low=0.9, high=1),
            Continuous("b", low=0.0, high=0.1),
            Continuous("eta", low=0.001, high=0.1),
        ],
        metric=HarmonicMean(
            10,
            Serendipity(),
            cornac.metrics.FMeasure(k=10),
            cornac.metrics.NCRR(),
            cornac.metrics.NDCG()
        ),
        eval_method=bm,
        n_trails=20,
    )

    # Put everything together into an experiment and run it
    experiment = cornac.Experiment(eval_method=bm, models=[rs_ctr], metrics=eval_metrics)
    experiment.run()

    # Print best params
    print('Random search best params: ', rs_ctr.best_params)

    EXPERIMENT = experiment

    if split_type == 'strat':
        TRAINING_SET = strat_train_df

    else:
        TRAINING_SET = lsuo_train_df

    # Evaluate and calculate harmonic mean
    evaluate(EXPERIMENT, TRAINING_SET)

def train_convMF(model_name, text_column, split_type, weighted_or_unweighted):

    if weighted_or_unweighted == 'unweighted':
        if split_type == 'strat':
            like_data_train = strat_like_data_train
            like_data_validation = strat_like_data_validation
            like_data_test = strat_like_data_test

        else:
            like_data_train = lsuo_like_data_train
            like_data_validation = lsuo_like_data_validation
            like_data_test = lsuo_like_data_test

    else:
        #junyi's modifications
        # train test val filenames
        trainfile = f'{split_type}_train'
        testfile = f'{split_type}_test'
        valfile = f'{split_type}_validation'

        # train test val datasets
        train = pd.read_csv('model_data/train_val_test/'+trainfile+'.csv')
        test = pd.read_csv('model_data/train_val_test/'+testfile+'.csv')
        val = pd.read_csv('model_data/train_val_test/'+valfile+'.csv')

        # main data dataset
        user = pd.read_csv('model_data/auxiliary/users.csv')
        cat = pd.read_csv('model_data/auxiliary/cats.csv')
        # interaction = pd.read_csv('model_data/auxiliary/interaction.csv')

        # list of datasets
        dfs = [train, test, val]
        for i in range(len(dfs)):
            # for all
            dfs[i]['like'] = dfs[i]['like'].apply(lambda x: 1 if x else 0)
            # train
            if i == 0:
                dfs[i]['click'] = dfs[i]['click'].apply(lambda x: 1 if x else 0)
                dfs[i].drop(['Unnamed: 0', 'id','created_at', 'updated_at'], axis=1, inplace=True)
                dfs[i] = dfs[i][['userID', 'catID', 'like', 'dwell_time_ms', 'click']]
            elif i == 1 or i == 2:
                dfs[i].drop(['Unnamed: 0', 'id', 'dwell_time_ms', 'click', 'created_at', 'updated_at'], axis=1, inplace=True)
                dfs[i] = dfs[i][['userID', 'catID', 'like']]
        train, test, val = dfs[0], dfs[1], dfs[2]
        # checking dwell_time_ms in seconds
        train['dwell_time_ms'].apply(lambda x: x/1000).describe()
        # applying log transformation on dwell time to reduce impact of outliers
        train['dwell_time_ms'].apply(lambda x: np.log(x)).describe()
        train['log_dwell_time'] = train['dwell_time_ms'].apply(lambda x: np.log(x))
        train.drop(['dwell_time_ms'], axis=1, inplace=True)
        scaler = MinMaxScaler()
        train['norm_log_dwell_time'] = scaler.fit_transform(train[['log_dwell_time']])
        train.drop(['log_dwell_time'], axis=1, inplace=True)
        train_weighted = train.copy()
        # weights L, C, D for like, click, dwell_time
        L = 0.5
        C = 0.25
        D = 0.25
        train_weighted['rating'] = L*train_weighted['like'] + C*train_weighted['click'] + D* train_weighted['norm_log_dwell_time']
        train_weighted.drop(['like','click','norm_log_dwell_time'], axis=1, inplace=True)
        # convert to tuple
        like_data_train = [tuple(train_weighted.iloc[i]) for i in range(len(train_weighted))]
        like_data_test = [tuple(test.iloc[i]) for i in range(len(test))]
        like_data_validation = [tuple(val.iloc[i]) for i in range(len(val))]

    if text_column == 'desc':
        cat_ids = desc_cat_ids
        texts = desc_texts

    else:
        cat_ids = details_cat_ids
        texts = details_texts


    # Instantiate a TextModality, it makes it convenient to work with text auxiliary information
    # For more details, please refer to the tutorial on how to work with auxiliary data
    item_text_modality = TextModality(
        corpus=texts,
        ids=cat_ids,
        tokenizer=BaseTokenizer(sep=" ", stop_words="english"),
        max_vocab=8000,
        max_doc_freq=0.5,
    )

    # Define an evaluation method to split feedback into train and test sets
    if split_type == 'strat':
        bm = CombinedBaseMethod.from_splits(
        train_data=like_data_train,
        val_data=like_data_validation,
        test_data=like_data_test,
        verbose=True,
        item_text=item_text_modality,
        exclude_unknowns=True,
        )

    else:
        bm = CombinedBaseMethod.from_splits(
            train_data=like_data_train,
            val_data=like_data_validation,
            test_data=like_data_test,
            verbose=True,
            item_text=item_text_modality,
            exclude_unknowns=False,
            )

    # Instantiate CTR model
    convMF = cornac.models.ConvMF(
        name=f'{model_name}_{text_column}_{split_type}_{weighted_or_unweighted}',
        n_epochs=50,
        cnn_epochs=5,
        verbose=False,
        seed=123
        )

    # Use these for evaluation
    eval_metrics = [
        HarmonicMean(
            10,
            Serendipity(),
            cornac.metrics.FMeasure(k=10),
            cornac.metrics.NCRR(),
            cornac.metrics.NDCG()
        ),
        Serendipity(),
        cornac.metrics.FMeasure(k=10),
        cornac.metrics.NCRR(),
        cornac.metrics.NDCG()
    ]

    # Wrap MF model inside RandomSearch along with the searching space, try 30 times
    rs_convMF = NewRandomSearch(
        model=convMF,
        space=[
            Discrete("k", [50, 75, 100, 150, 200]),
            Discrete("cnn_bs", [16, 32, 64, 128]),
            Continuous("cnn_lr", low=1e-4, high=1e-2),
            Continuous("lambda_u", low=1e-1, high=1e2),
            Continuous("lambda_v", low=1e2, high=1e4),
            Continuous("dropout_rate", low=0.05, high=0.15),
        ],
        metric=HarmonicMean(
            10,
            Serendipity(),
            cornac.metrics.FMeasure(k=10),
            cornac.metrics.NCRR(),
            cornac.metrics.NDCG()
        ),
        eval_method=bm,
        n_trails=20,
    )

    # Put everything together into an experiment and run it
    experiment = cornac.Experiment(eval_method=bm, models=[rs_convMF], metrics=eval_metrics)
    experiment.run()

    # Print best params
    print('Random search best params: ', rs_convMF.best_params)

    EXPERIMENT = experiment

    if split_type == 'strat':
        TRAINING_SET = strat_train_df

    else:
        TRAINING_SET = lsuo_train_df

    # Evaluate and calculate harmonic mean
    evaluate(EXPERIMENT, TRAINING_SET)

In [6]:
# CTR desc stratified split (unweighted)

model_name = 'CTR'
text_column = 'description'
split_type = 'strat'
weighted_or_unweighted = 'unweighted'

train_ctr(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_CTR_description_strat_unweighted] Training started!
Evaluating: {'a': 0.9006614738389299, 'b': 0.045161204318163665, 'eta': 0.05384915979390073, 'k': 150, 'lambda_u': 7.307317227823263, 'lambda_v': 5.137607158186093}


100%|██████████| 200/200 [08:10<00:00,  2.45s/it, cf_loss=525, lda_likelihood=-2.66e+4]


Learning completed!
Evaluating: {'a': 0.9008246600981573, 'b': 0.04095020222256599, 'eta': 0.055762965973127375, 'k': 100, 'lambda_u': 4.273549540145383, 'lambda_v': 4.731855282599306}


100%|██████████| 200/200 [03:37<00:00,  1.09s/it, cf_loss=438, lda_likelihood=-2.09e+4]


Learning completed!
Evaluating: {'a': 0.9583877670084683, 'b': 0.039235880612214946, 'eta': 0.07145140463021744, 'k': 100, 'lambda_u': 8.40151897862538, 'lambda_v': 6.0537808001396245}


...

Learning completed!
Evaluating: {'a': 0.910164085243495, 'b': 0.06480584696985309, 'eta': 0.03917335227873423, 'k': 100, 'lambda_u': 4.18420386658643, 'lambda_v': 7.802802525466104}


100%|██████████| 200/200 [03:01<00:00,  1.10it/s, cf_loss=483, lda_likelihood=-1.8e+4] 


Learning completed!
Best parameter settings: {'a': 0.9015794736606453, 'b': 0.027018142044926943, 'eta': 0.05100096844728663, 'k': 50, 'lambda_u': 2.8690381441484663, 'lambda_v': 2.6409219328165734}
HarmonicMean = 0.0076

[RandomSearch_CTR_description_strat_unweighted] Evaluation started!


Ranking: 100%|██████████| 96/96 [00:03<00:00, 31.48it/s]



VALIDATION:
...
                                              |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
--------------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_CTR_description_strat_unweighted | 0.0243 |       0.0076 |  0.0546 |  0.2440 |      0.0091 |   3.0516

TEST:
...
                                              |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
--------------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_CTR_description_strat_unweighted | 0.0136 |       0.0060 |  0.0439 |  0.2301 |      0.0084 | 3933.5666 |   3.5755

Random search best params:  {'a': 0.9015794736606453, 'b': 0.027018142044926943, 'eta': 0.05100096844728663, 'k': 50, 'lambda_u': 2.8690381441484663, 'lambda_v': 2.6409219328165734}
RandomSearch_CTR_description_strat_unweighted
F1@10: 0.014
NCRR

In [7]:
# CTR details stratified split (unweighted)

model_name = 'CTR'
text_column = 'details'
split_type = 'strat'
weighted_or_unweighted = 'unweighted'

train_ctr(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_CTR_details_strat_unweighted] Training started!
Evaluating: {'a': 0.9111526519689233, 'b': 0.025581379309709707, 'eta': 0.048363768721040055, 'k': 100, 'lambda_u': 7.963422530706351, 'lambda_v': 1.0456859807378849}


100%|██████████| 200/200 [02:55<00:00,  1.14it/s, cf_loss=378, lda_likelihood=-1.95e+4]


Learning completed!
Evaluating: {'a': 0.9659223825338872, 'b': 0.024668209623546978, 'eta': 0.027572885944642418, 'k': 100, 'lambda_u': 3.3392638327054947, 'lambda_v': 5.6074934646162475}


100%|██████████| 200/200 [03:00<00:00,  1.11it/s, cf_loss=423, lda_likelihood=-1.8e+4] 


Learning completed!
Evaluating: {'a': 0.9481327908756115, 'b': 0.03475198037086957, 'eta': 0.05991552680901714, 'k': 100, 'lambda_u': 6.641308606550052, 'lambda_v': 2.214310247290354}


...

Learning completed!
Evaluating: {'a': 0.963145189194876, 'b': 0.09670826848023534, 'eta': 0.01680921584451462, 'k': 50, 'lambda_u': 6.0122354495657895, 'lambda_v': 3.8313089127468123}


100%|██████████| 200/200 [01:12<00:00,  2.76it/s, cf_loss=496, lda_likelihood=-1.03e+4]


Learning completed!
Best parameter settings: {'a': 0.9734401452135346, 'b': 0.0020241128979698432, 'eta': 0.05902919129983911, 'k': 50, 'lambda_u': 5.106142948899999, 'lambda_v': 6.480866707600999}
HarmonicMean = 0.0091

[RandomSearch_CTR_details_strat_unweighted] Evaluation started!


Ranking: 100%|██████████| 96/96 [00:03<00:00, 31.61it/s]



VALIDATION:
...
                                          |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
----------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_CTR_details_strat_unweighted | 0.0375 |       0.0091 |  0.0659 |  0.2565 |      0.0091 |   3.0385

TEST:
...
                                          |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
----------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_CTR_details_strat_unweighted | 0.0203 |       0.0073 |  0.0493 |  0.2331 |      0.0084 | 4443.8769 |   3.6446

Random search best params:  {'a': 0.9734401452135346, 'b': 0.0020241128979698432, 'eta': 0.05902919129983911, 'k': 50, 'lambda_u': 5.106142948899999, 'lambda_v': 6.480866707600999}
RandomSearch_CTR_details_strat_unweighted
F1@10: 0.020
NCRR: 0.049
NDCG: 0.233
Distribut

In [8]:
# CTR desc lsuo split (unweighted)

model_name = 'CTR'
text_column = 'desc'
split_type = 'lsuo'
weighted_or_unweighted = 'unweighted'

train_ctr(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[RandomSearch_CTR_desc_lsuo_unweighted] Training started!
Evaluating: {'a': 0.9581075989483444, 'b': 0.09675009673394318, 'eta': 0.08507355692253224, 'k': 75, 'lambda_u': 6.118130487254779, 'lambda_v': 9.86423412093533}


100%|██████████| 200/200 [01:33<00:00,  2.14it/s, cf_loss=473, lda_likelihood=-928]    


Learning completed!
Evaluating: {'a': 0.919267490328438, 'b': 0.03906913809108864, 'eta': 0.0167098349809198, 'k': 150, 'lambda_u': 7.89015327826145, 'lambda_v': 6.756674585065074}


100%|██████████| 200/200 [06:24<00:00,  1.92s/it, cf_loss=458, lda_likelihood=-1.16e+3]


Learning completed!
Evaluating: {'a': 0.9275445372515654, 'b': 0.057305581215865944, 'eta': 0.06032411835958567, 'k': 150, 'lambda_u': 3.5574680775747836, 'lambda_v': 1.3241900753684985}


...

Learning completed!
Evaluating: {'a': 0.9474531081040253, 'b': 0.019666692177224256, 'eta': 0.01836838242858228, 'k': 50, 'lambda_u': 6.583725504901151, 'lambda_v': 8.737429271035834}


100%|██████████| 200/200 [01:04<00:00,  3.11it/s, cf_loss=440, lda_likelihood=-735]   


Learning completed!
Best parameter settings: {'a': 0.9581075989483444, 'b': 0.09675009673394318, 'eta': 0.08507355692253224, 'k': 75, 'lambda_u': 6.118130487254779, 'lambda_v': 9.86423412093533}
HarmonicMean = 0.0000

[RandomSearch_CTR_desc_lsuo_unweighted] Evaluation started!


Ranking: 100%|██████████| 10/10 [00:00<00:00, 40.45it/s]



VALIDATION:
...
                                      |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_CTR_desc_lsuo_unweighted | 0.0428 |       0.0000 |  0.1674 |  0.4395 |      0.0000 |   0.2487

TEST:
...
                                      |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_CTR_desc_lsuo_unweighted | 0.0246 |       0.0000 |  0.0997 |  0.3810 |      0.0000 | 4704.3239 |   0.2257

Random search best params:  {'a': 0.9581075989483444, 'b': 0.09675009673394318, 'eta': 0.08507355692253224, 'k': 75, 'lambda_u': 6.118130487254779, 'lambda_v': 9.86423412093533}
RandomSearch_CTR_desc_lsuo_unweighted
F1@10: 0.025
NCRR: 0.100
NDCG: 0.381
Distributional coverage: 8.616
Serendipi

In [9]:
# CTR details lsuo split (unweighted)

model_name = 'CTR'
text_column = 'details'
split_type = 'lsuo'
weighted_or_unweighted = 'unweighted'

train_ctr(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[RandomSearch_CTR_details_lsuo_unweighted] Training started!
Evaluating: {'a': 0.9471075762082883, 'b': 0.010975604292272602, 'eta': 0.045020164335017876, 'k': 150, 'lambda_u': 6.469055224521553, 'lambda_v': 9.75329612659715}


100%|██████████| 200/200 [05:13<00:00,  1.57s/it, cf_loss=540, lda_likelihood=-2.44e+4]


Learning completed!
Evaluating: {'a': 0.9476330636655335, 'b': 0.06011623586550307, 'eta': 0.010958758766982782, 'k': 50, 'lambda_u': 1.3923527755156808, 'lambda_v': 0.6995978332179038}


100%|██████████| 200/200 [01:08<00:00,  2.93it/s, cf_loss=174, lda_likelihood=-1.29e+4]


Learning completed!
Evaluating: {'a': 0.980870067176476, 'b': 0.05363042459501513, 'eta': 0.0027396295230787925, 'k': 50, 'lambda_u': 1.0071517779647114, 'lambda_v': 5.376511703899376}


...

Learning completed!
Evaluating: {'a': 0.9867583934552846, 'b': 0.009019676529770005, 'eta': 0.09008241102168979, 'k': 200, 'lambda_u': 6.153938830748407, 'lambda_v': 0.4603713211316615}


100%|██████████| 200/200 [07:45<00:00,  2.33s/it, cf_loss=253, lda_likelihood=-2.91e+4]


Learning completed!
Best parameter settings: {'a': 0.9471075762082883, 'b': 0.010975604292272602, 'eta': 0.045020164335017876, 'k': 150, 'lambda_u': 6.469055224521553, 'lambda_v': 9.75329612659715}
HarmonicMean = 0.0000

[RandomSearch_CTR_details_lsuo_unweighted] Evaluation started!


Ranking: 100%|██████████| 10/10 [00:00<00:00, 39.73it/s]



VALIDATION:
...
                                         |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
---------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_CTR_details_lsuo_unweighted | 0.0428 |       0.0000 |  0.1674 |  0.4395 |      0.0000 |   0.2537

TEST:
...
                                         |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
---------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_CTR_details_lsuo_unweighted | 0.0246 |       0.0000 |  0.0997 |  0.3810 |      0.0000 | 4023.5976 |   0.2252

Random search best params:  {'a': 0.9471075762082883, 'b': 0.010975604292272602, 'eta': 0.045020164335017876, 'k': 150, 'lambda_u': 6.469055224521553, 'lambda_v': 9.75329612659715}
RandomSearch_CTR_details_lsuo_unweighted
F1@10: 0.025
NCRR: 0.100
NDCG: 0.381
Distributional c

In [10]:
# CTR desc stratified split (weighted)

model_name = 'CTR'
text_column = 'desc'
split_type = 'strat'
weighted_or_unweighted = 'weighted'

train_ctr(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_CTR_desc_strat_weighted] Training started!
Evaluating: {'a': 0.9493301691872464, 'b': 0.002178838925403326, 'eta': 0.013127260146536174, 'k': 200, 'lambda_u': 8.048052341229711, 'lambda_v': 8.525799636611234}


100%|██████████| 200/200 [07:36<00:00,  2.28s/it, cf_loss=246, lda_likelihood=-1.26e+3]


Learning completed!
Evaluating: {'a': 0.9468610308061826, 'b': 0.06882895102129218, 'eta': 0.03406018916009659, 'k': 50, 'lambda_u': 9.250010222576968, 'lambda_v': 7.030257724217479}


100%|██████████| 200/200 [01:05<00:00,  3.04it/s, cf_loss=260, lda_likelihood=-782]    


Learning completed!
Evaluating: {'a': 0.9223183237085391, 'b': 0.09358304891681923, 'eta': 0.08053974527992168, 'k': 200, 'lambda_u': 1.9584977087301252, 'lambda_v': 7.185875017627006}


...

Learning completed!
Evaluating: {'a': 0.9471921815995236, 'b': 0.08528572665252962, 'eta': 0.053292750893809616, 'k': 200, 'lambda_u': 1.738481593820751, 'lambda_v': 2.749726930009925}


100%|██████████| 200/200 [08:40<00:00,  2.60s/it, cf_loss=162, lda_likelihood=-1.49e+3]


Learning completed!
Best parameter settings: {'a': 0.9198401490116883, 'b': 0.010096977542049947, 'eta': 0.03584190602884217, 'k': 150, 'lambda_u': 3.3993292349322277, 'lambda_v': 6.153824748975314}
HarmonicMean = 0.0117

[RandomSearch_CTR_desc_strat_weighted] Evaluation started!


Ranking: 100%|██████████| 96/96 [00:03<00:00, 30.87it/s]



VALIDATION:
...
                                     |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
------------------------------------ + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_CTR_desc_strat_weighted | 0.0439 |       0.0117 |  0.0774 |  0.2689 |      0.0091 |   3.1121

TEST:
...
                                     |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
------------------------------------ + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_CTR_desc_strat_weighted | 0.0280 |       0.0089 |  0.0636 |  0.2529 |      0.0084 | 5726.0769 |   3.7105

Random search best params:  {'a': 0.9198401490116883, 'b': 0.010096977542049947, 'eta': 0.03584190602884217, 'k': 150, 'lambda_u': 3.3993292349322277, 'lambda_v': 6.153824748975314}
RandomSearch_CTR_desc_strat_weighted
F1@10: 0.028
NCRR: 0.064
NDCG: 0.253
Distributional coverage: 8.633
Serendipity:

In [11]:
# CTR details stratified split (weighted)

model_name = 'CTR'
text_column = 'details'
split_type = 'strat'
weighted_or_unweighted = 'weighted'

train_ctr(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_CTR_details_strat_weighted] Training started!
Evaluating: {'a': 0.956334100776738, 'b': 0.027335056966866823, 'eta': 0.05509559897436499, 'k': 150, 'lambda_u': 8.97018797082085, 'lambda_v': 1.9184472979942864}


100%|██████████| 200/200 [05:36<00:00,  1.68s/it, cf_loss=251, lda_likelihood=-2.56e+4]


Learning completed!
Evaluating: {'a': 0.9677922571346613, 'b': 0.041403498974021326, 'eta': 0.09553992728879279, 'k': 150, 'lambda_u': 1.2169543598726134, 'lambda_v': 1.1761464848223422}


100%|██████████| 200/200 [05:36<00:00,  1.68s/it, cf_loss=112, lda_likelihood=-2.72e+4]


Learning completed!
Evaluating: {'a': 0.9257127228486499, 'b': 0.0031297162420985906, 'eta': 0.09114589350112817, 'k': 75, 'lambda_u': 5.052270617165201, 'lambda_v': 9.966811669180712}


...

Learning completed!
Evaluating: {'a': 0.9620149754295239, 'b': 0.05321407541286688, 'eta': 0.05337687743403453, 'k': 50, 'lambda_u': 1.8304352628346727, 'lambda_v': 4.950123579912475}


100%|██████████| 200/200 [01:11<00:00,  2.82it/s, cf_loss=181, lda_likelihood=-1.03e+4]


Learning completed!
Best parameter settings: {'a': 0.9950611128858565, 'b': 0.04662737769583036, 'eta': 0.0023846855603643817, 'k': 75, 'lambda_u': 6.241999295694586, 'lambda_v': 7.886977653698045}
HarmonicMean = 0.0074

[RandomSearch_CTR_details_strat_weighted] Evaluation started!


Ranking: 100%|██████████| 96/96 [00:02<00:00, 32.28it/s]



VALIDATION:
...
                                        |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
--------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_CTR_details_strat_weighted | 0.0240 |       0.0074 |  0.0556 |  0.2462 |      0.0091 |   2.9764

TEST:
...
                                        |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
--------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_CTR_details_strat_weighted | 0.0112 |       0.0055 |  0.0373 |  0.2261 |      0.0084 | 4792.3726 |   3.5967

Random search best params:  {'a': 0.9950611128858565, 'b': 0.04662737769583036, 'eta': 0.0023846855603643817, 'k': 75, 'lambda_u': 6.241999295694586, 'lambda_v': 7.886977653698045}
RandomSearch_CTR_details_strat_weighted
F1@10: 0.011
NCRR: 0.037
NDCG: 0.226
Distributional coverage

In [12]:
# CTR desc lsuo split (weighted)

model_name = 'CTR'
text_column = 'desc'
split_type = 'lsuo'
weighted_or_unweighted = 'weighted'

train_ctr(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[RandomSearch_CTR_desc_lsuo_weighted] Training started!
Evaluating: {'a': 0.9950540186794792, 'b': 0.00380599591230657, 'eta': 0.021637294692872348, 'k': 50, 'lambda_u': 0.30039278798666413, 'lambda_v': 9.387831746890065}


100%|██████████| 200/200 [01:02<00:00,  3.20it/s, cf_loss=65.8, lda_likelihood=-882]    


Learning completed!
Evaluating: {'a': 0.9762874106406632, 'b': 0.014535003465421116, 'eta': 0.007970275176039987, 'k': 100, 'lambda_u': 8.430143088936461, 'lambda_v': 6.700356925479086}


100%|██████████| 200/200 [02:52<00:00,  1.16it/s, cf_loss=236, lda_likelihood=-1.04e+3]


Learning completed!
Evaluating: {'a': 0.9074181461296489, 'b': 0.0962016458980852, 'eta': 0.05145803818589798, 'k': 50, 'lambda_u': 7.126780711649708, 'lambda_v': 9.901898545497177}


...

Learning completed!
Evaluating: {'a': 0.900439263770219, 'b': 0.0846994223105857, 'eta': 0.09543854071824463, 'k': 200, 'lambda_u': 4.794623438514491, 'lambda_v': 7.58691044767803}


100%|██████████| 200/200 [07:37<00:00,  2.29s/it, cf_loss=217, lda_likelihood=-1.37e+3]


Learning completed!
Best parameter settings: {'a': 0.9950540186794792, 'b': 0.00380599591230657, 'eta': 0.021637294692872348, 'k': 50, 'lambda_u': 0.30039278798666413, 'lambda_v': 9.387831746890065}
HarmonicMean = 0.0000

[RandomSearch_CTR_desc_lsuo_weighted] Evaluation started!


Ranking: 100%|██████████| 10/10 [00:00<00:00, 34.39it/s]



VALIDATION:
...
                                    |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
----------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_CTR_desc_lsuo_weighted | 0.0428 |       0.0000 |  0.1674 |  0.4395 |      0.0000 |   0.2927

TEST:
...
                                    |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
----------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_CTR_desc_lsuo_weighted | 0.0246 |       0.0000 |  0.0997 |  0.3810 |      0.0000 | 4499.1473 |   0.2247

Random search best params:  {'a': 0.9950540186794792, 'b': 0.00380599591230657, 'eta': 0.021637294692872348, 'k': 50, 'lambda_u': 0.30039278798666413, 'lambda_v': 9.387831746890065}
RandomSearch_CTR_desc_lsuo_weighted
F1@10: 0.025
NCRR: 0.100
NDCG: 0.381
Distributional coverage: 8.616
Serendipity: 0.790


In [13]:
# CTR details lsuo split (weighted)

model_name = 'CTR'
text_column = 'details'
split_type = 'lsuo'
weighted_or_unweighted = 'weighted'

train_ctr(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[RandomSearch_CTR_details_lsuo_weighted] Training started!
Evaluating: {'a': 0.9292452087874831, 'b': 0.05999875607312708, 'eta': 0.07239179647674573, 'k': 75, 'lambda_u': 8.520380603047805, 'lambda_v': 3.573772634107786}


100%|██████████| 200/200 [01:48<00:00,  1.84it/s, cf_loss=258, lda_likelihood=-1.69e+4]


Learning completed!
Evaluating: {'a': 0.9223029151049579, 'b': 0.030973916181506612, 'eta': 0.07062065165383528, 'k': 50, 'lambda_u': 7.748395039217611, 'lambda_v': 8.345119029676349}


100%|██████████| 200/200 [01:08<00:00,  2.94it/s, cf_loss=275, lda_likelihood=-1.08e+4]


Learning completed!
Evaluating: {'a': 0.9053041652602726, 'b': 0.042926977990113926, 'eta': 0.09338557319665378, 'k': 50, 'lambda_u': 9.9079555810639, 'lambda_v': 4.260272088653481}


...

Learning completed!
Evaluating: {'a': 0.9456871499864848, 'b': 0.06796645312036831, 'eta': 0.03221113155657776, 'k': 200, 'lambda_u': 8.55620036633677, 'lambda_v': 0.09699298918391054}


100%|██████████| 200/200 [07:43<00:00,  2.32s/it, cf_loss=109, lda_likelihood=-2.73e+4]


Learning completed!
Best parameter settings: {'a': 0.9292452087874831, 'b': 0.05999875607312708, 'eta': 0.07239179647674573, 'k': 75, 'lambda_u': 8.520380603047805, 'lambda_v': 3.573772634107786}
HarmonicMean = 0.0000

[RandomSearch_CTR_details_lsuo_weighted] Evaluation started!


Ranking: 100%|██████████| 10/10 [00:00<00:00, 40.37it/s]



VALIDATION:
...
                                       |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
-------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_CTR_details_lsuo_weighted | 0.0428 |       0.0000 |  0.1674 |  0.4395 |      0.0000 |   0.2497

TEST:
...
                                       |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
-------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_CTR_details_lsuo_weighted | 0.0246 |       0.0000 |  0.0997 |  0.3810 |      0.0000 | 4102.3977 |   0.2267

Random search best params:  {'a': 0.9292452087874831, 'b': 0.05999875607312708, 'eta': 0.07239179647674573, 'k': 75, 'lambda_u': 8.520380603047805, 'lambda_v': 3.573772634107786}
RandomSearch_CTR_details_lsuo_weighted
F1@10: 0.025
NCRR: 0.100
NDCG: 0.381
Distributional coverage: 8.616
S

In [14]:
# convMF desc stratified split (unweighted) NOTE

model_name = 'convMF'
text_column = 'desc'
split_type = 'strat'
weighted_or_unweighted = 'unweighted'

train_convMF(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_convMF_desc_strat_unweighted] Training started!

[RandomSearch_convMF_desc_strat_unweighted] Evaluation started!


Ranking: 100%|██████████| 96/96 [00:03<00:00, 31.95it/s]



VALIDATION:
...
                                          |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
----------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_convMF_desc_strat_unweighted | 0.0349 |       0.0095 |  0.0794 |  0.2565 |      0.0091 |   3.0061

TEST:
...
                                          |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
----------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_convMF_desc_strat_unweighted | 0.0387 |       0.0092 |  0.0673 |  0.2486 |      0.0084 | 8584.9833 |   3.6936

Random search best params:  {'cnn_bs': 64, 'cnn_lr': 0.003800634125320658, 'dropout_rate': 0.07345128751822862, 'k': 200, 'lambda_u': 63.13451477306333, 'lambda_v': 1011.8389054562443}
RandomSearch_convMF_desc_strat_unweighted
F1@10: 0.039
NCRR: 0.067
NDCG: 0.249
Distr

In [15]:
# convMF details stratified split (unweighted)

model_name = 'convMF'
text_column = 'details'
split_type = 'strat'
weighted_or_unweighted = 'unweighted'

train_convMF(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_convMF_details_strat_unweighted] Training started!

[RandomSearch_convMF_details_strat_unweighted] Evaluation started!


Ranking: 100%|██████████| 96/96 [00:03<00:00, 30.57it/s]



VALIDATION:
...
                                             |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
-------------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_convMF_details_strat_unweighted | 0.0320 |       0.0085 |  0.0588 |  0.2412 |      0.0091 |   3.1432

TEST:
...
                                             |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
-------------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_convMF_details_strat_unweighted | 0.0131 |       0.0041 |  0.0478 |  0.2302 |      0.0084 | 8934.8317 |   3.8533

Random search best params:  {'cnn_bs': 16, 'cnn_lr': 0.00037702135492519493, 'dropout_rate': 0.06739065162367008, 'k': 100, 'lambda_u': 31.294896174949283, 'lambda_v': 4320.8779389318015}
RandomSearch_convMF_details_strat_unweighted
F1@10: 0.013
NCRR:

In [16]:
# convMF desc lsuo split (unweighted)

model_name = 'convMF'
text_column = 'desc'
split_type = 'lsuo'
weighted_or_unweighted = 'unweighted'

train_convMF(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[RandomSearch_convMF_desc_lsuo_unweighted] Training started!

[RandomSearch_convMF_desc_lsuo_unweighted] Evaluation started!


Ranking: 100%|██████████| 10/10 [00:00<00:00, 40.95it/s]



VALIDATION:
...
                                         |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
---------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_convMF_desc_lsuo_unweighted | 0.0428 |       0.0000 |  0.1674 |  0.4395 |      0.0000 |   0.2457

TEST:
...
                                         |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
---------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_convMF_desc_lsuo_unweighted | 0.0246 |       0.0000 |  0.0997 |  0.3810 |      0.0000 | 8817.4722 |   0.2372

Random search best params:  {'cnn_bs': 64, 'cnn_lr': 0.00715825767317995, 'dropout_rate': 0.09284709261868257, 'k': 100, 'lambda_u': 55.17634543138084, 'lambda_v': 7222.742800877075}
RandomSearch_convMF_desc_lsuo_unweighted
F1@10: 0.025
NCRR: 0.100
NDCG: 0.381
Distributional

In [17]:
# convMF details lsuo split (unweighted)

model_name = 'convMF'
text_column = 'details'
split_type = 'lsuo'
weighted_or_unweighted = 'unweighted'

train_convMF(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.5
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[RandomSearch_convMF_details_lsuo_unweighted] Training started!

[RandomSearch_convMF_details_lsuo_unweighted] Evaluation started!


Ranking: 100%|██████████| 10/10 [00:00<00:00, 36.60it/s]



VALIDATION:
...
                                            |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
------------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_convMF_details_lsuo_unweighted | 0.0428 |       0.0000 |  0.1674 |  0.4395 |      0.0000 |   0.2752

TEST:
...
                                            |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
------------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_convMF_details_lsuo_unweighted | 0.0246 |       0.0000 |  0.0997 |  0.3810 |      0.0000 | 9694.8496 |   0.2682

Random search best params:  {'cnn_bs': 64, 'cnn_lr': 0.00715825767317995, 'dropout_rate': 0.09284709261868257, 'k': 100, 'lambda_u': 55.17634543138084, 'lambda_v': 7222.742800877075}
RandomSearch_convMF_details_lsuo_unweighted
F1@10: 0.025
NCRR: 0.100
NDCG:

In [18]:
# convMF desc stratified split (weighted)

model_name = 'convMF'
text_column = 'desc'
split_type = 'strat'
weighted_or_unweighted = 'weighted'

train_convMF(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_convMF_desc_strat_weighted] Training started!

[RandomSearch_convMF_desc_strat_weighted] Evaluation started!


Ranking: 100%|██████████| 96/96 [00:03<00:00, 31.73it/s]



VALIDATION:
...
                                        |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
--------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_convMF_desc_strat_weighted | 0.0334 |       0.0083 |  0.0780 |  0.2532 |      0.0091 |   3.0266

TEST:
...
                                        |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
--------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_convMF_desc_strat_weighted | 0.0200 |       0.0063 |  0.0398 |  0.2140 |      0.0084 | 8545.9508 |   3.6761

Random search best params:  {'cnn_bs': 16, 'cnn_lr': 0.00014093296321896555, 'dropout_rate': 0.11380225178789184, 'k': 200, 'lambda_u': 4.398388002820375, 'lambda_v': 8763.006231160176}
RandomSearch_convMF_desc_strat_weighted
F1@10: 0.020
NCRR: 0.040
NDCG: 0.214
Distributional cov

In [19]:
# convMF details stratified split (weighted)

model_name = 'convMF'
text_column = 'details'
split_type = 'strat'
weighted_or_unweighted = 'weighted'

train_convMF(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 104
Number of items = 400
Number of ratings = 3874
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 100
Number of items = 195
Number of ratings = 487
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 96
Number of items = 184
Number of ratings = 476
---
Total users = 104
Total items = 400

[RandomSearch_convMF_details_strat_weighted] Training started!

[RandomSearch_convMF_details_strat_weighted] Evaluation started!


Ranking: 100%|██████████| 96/96 [00:03<00:00, 31.99it/s]



VALIDATION:
...
                                           |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
------------------------------------------ + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_convMF_details_strat_weighted | 0.0239 |       0.0076 |  0.0696 |  0.2370 |      0.0091 |   3.0023

TEST:
...
                                           |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
------------------------------------------ + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_convMF_details_strat_weighted | 0.0195 |       0.0065 |  0.0510 |  0.2169 |      0.0084 | 7680.6526 |   3.6646

Random search best params:  {'cnn_bs': 64, 'cnn_lr': 0.003800634125320658, 'dropout_rate': 0.07345128751822862, 'k': 200, 'lambda_u': 63.13451477306333, 'lambda_v': 1011.8389054562443}
RandomSearch_convMF_details_strat_weighted
F1@10: 0.020
NCRR: 0.051
NDCG: 0.21

In [20]:
# convMF desc lsuo split (weighted)

model_name = 'convMF'
text_column = 'desc'
split_type = 'lsuo'
weighted_or_unweighted = 'weighted'

train_convMF(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[RandomSearch_convMF_desc_lsuo_weighted] Training started!

[RandomSearch_convMF_desc_lsuo_weighted] Evaluation started!


Ranking: 100%|██████████| 10/10 [00:00<00:00, 39.97it/s]



VALIDATION:
...
                                       |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
-------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_convMF_desc_lsuo_weighted | 0.0428 |       0.0000 |  0.1674 |  0.4395 |      0.0000 |   0.2522

TEST:
...
                                       |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
-------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_convMF_desc_lsuo_weighted | 0.0246 |       0.0000 |  0.0997 |  0.3810 |      0.0000 | 7457.9751 |   0.2277

Random search best params:  {'cnn_bs': 64, 'cnn_lr': 0.00715825767317995, 'dropout_rate': 0.09284709261868257, 'k': 100, 'lambda_u': 55.17634543138084, 'lambda_v': 7222.742800877075}
RandomSearch_convMF_desc_lsuo_weighted
F1@10: 0.025
NCRR: 0.100
NDCG: 0.381
Distributional coverage: 8.6

In [21]:
# convMF details lsuo split (weighted)

model_name = 'convMF'
text_column = 'details'
split_type = 'lsuo'
weighted_or_unweighted = 'weighted'

train_convMF(model_name=model_name,
          text_column=text_column,
          split_type=split_type,
          weighted_or_unweighted=weighted_or_unweighted
          )

creating from splits
initialising Combined Base
rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 83
Number of items = 397
Number of ratings = 3720
Max rating = 1.0
Min rating = 0.0
Global mean = 0.4
---
Test data:
Number of users = 11
Number of items = 101
Number of ratings = 458
Number of unknown users = 11
Number of unknown items = 0
---
Validation data:
Number of users = 10
Number of items = 308
Number of ratings = 663
---
Total users = 104
Total items = 403

[RandomSearch_convMF_details_lsuo_weighted] Training started!

[RandomSearch_convMF_details_lsuo_weighted] Evaluation started!


Ranking: 100%|██████████| 10/10 [00:00<00:00, 39.97it/s]



VALIDATION:
...
                                          |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Time (s)
----------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------
RandomSearch_convMF_details_lsuo_weighted | 0.0428 |       0.0000 |  0.1674 |  0.4395 |      0.0000 |   0.2522

TEST:
...
                                          |  F1@10 | HarmonicMean | NCRR@-1 | NDCG@-1 | Serendipity | Train (s) | Test (s)
----------------------------------------- + ------ + ------------ + ------- + ------- + ----------- + --------- + --------
RandomSearch_convMF_details_lsuo_weighted | 0.0246 |       0.0000 |  0.0997 |  0.3810 |      0.0000 | 7883.7402 |   0.2362

Random search best params:  {'cnn_bs': 64, 'cnn_lr': 0.00715825767317995, 'dropout_rate': 0.09284709261868257, 'k': 100, 'lambda_u': 55.17634543138084, 'lambda_v': 7222.742800877075}
RandomSearch_convMF_details_lsuo_weighted
F1@10: 0.025
NCRR: 0.100
NDCG: 0.381
Distrib